# Orca 2 on Colab
<a target="_blank" href="https://colab.research.google.com/github/antonio-f/Orca2/blob/main/Orca2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

The following contents are loosely based on the article "Orca 2: Teaching Small Language Models How to Reason" by A. Mitra et al. ([link](https://arxiv.org/abs/2311.11045)).

**Orca 2** is a language model from Microsoft Research. It builds on the previous Orca model and aims to explore the capabilities of smaller language models (LMs) through improved training signals and methods. There are two versions, one with 7 billion parameters and the other with 13 billion parameters. The goal is to demonstrate that smaller LMs, typically with around 10 billion parameters or less, can achieve enhanced reasoning abilities comparable to much larger models. Orca 2 outperforms models of similar size, including the original Orca, and exhibits performance levels similar to or better than models 5-10 times larger. The evaluation is based on complex tasks testing advanced reasoning abilities in zero-shot settings. The models are fine-tuned on high-quality synthetic data, and the weights of Orca 2 are made publicly available to encourage further research on the development, evaluation, and alignment of smaller LMs.

Orca 2 is designed with the understanding that diverse tasks require different solution strategies. Unlike a one-size-fits-all approach, it recognizes that while larger models like GPT-4 may excel at direct answers for complex tasks, smaller models can benefit from breaking tasks into steps.

In Orca 2, like its predecessor Orca 1, more advanced Language Model (LLMs) are employed to showcase various reasoning strategies across different tasks. However, in Orca 2, these strategies are specifically tailored to the task at hand. To generate nuanced data, the proficient LLM is provided with intricate prompts designed to elicit specific strategic behaviors, leading to more accurate results. During the training phase, the smaller model is exposed solely to the task and its resulting behavior, without access to the original prompts that triggered such behavior, enhancing the model's ability to generalize and adapt its reasoning capabilities.

### From Instruction Tuning to Explanation Tuning

Instruction tuning involves learning from input-output pairs where the input is natural language task description,and the output is a demonstration of the desired behavior. A weakness in instruction tuning is that a student model may generate outputs that are stylistically correct but ultimately incorrect. To address this issue, the authors introduce Explanation Tuning in Orca 1. This approach involves training student models on richer and more expressive reasoning signals obtained through `system instructions`. These instructions are crafted to elicit detailed explanations from a teacher model as it reasons through a task.

The process of explanation tuning initiates by creating a collection of N manually crafted `system instructions` aimed at prompting careful reasoning. These instructions, such as "think step-by-step" and "generate detailed answers," are designed to elicit thoughtful responses from advanced language models like GPT-4, emphasizing "Slow Thinking." These `system instructions` are integrated with user prompts spanning a wide range of tasks, forming a dataset of triplets composed of `system instruction`, `user prompt`, and the corresponding language model `LLM answer`. The student model is then trained to predict the LLM answer based on the given system instruction and user prompt.


### Teaching Orca 2 to be a "Cautious Reasoner"

An example of `system instructions` is the following:

```
You will be given a task. Use the following steps to solve it.
1. Identify the main theme or topic of the story.
2. Look for any cause and effect relationships between the sentences.
3. Find the sentence that could be the start of the story. Go through each of the answer
choices and analyze to figure it out.
4. Rearrange the sentences in the correct order based on the information gathered in
the previous steps.
5. Final answer: Write down the correct order of the sentences using their numbers,
such as ‘23415’.
```

Hence this instructions are small aids to conduct a sort of step-by-step procedure hoping to direct towards a correct answer. It's about breaking down a big problem into multiple subproblems.

Despite the correctness of all the provided answers, the question remains: which is the best answer for training the smaller model? This is a key issue and it is argued that smaller models should be taught to select the most effective solution strategy based on the problem at hand.
For example, GPT-4, being a larger model, can effortlessly produce a direct response, whereas a smaller model might not possess this capability and could necessitate an alternative approach, such as a step-by-step thought process. Consequently, instructing a smaller model to simply "mimic" the reasoning behavior of a more powerful counterpart may not be the optimal approach.

The term **Cautious Reasoning** is used to describe the process of determining the appropriate solution strategy for a given task. This choice is made among options such as direct answer generation or various 'Slow Thinking' strategies (such as step-by-step, guess and check, explain-then-answer, etc.)

The following illustrates the process of training a Cautious Reasoning LLM:
1. Start with a collection of diverse tasks
2. Guided by the performance of Orca, decide which tasks require which solution
strategy (e.g. direct-answer, step-by-step, explain-then-answer, etc.)
3. Write task-specific `system instruction(s)` corresponding to the chosen strategy
in order to obtain teacher responses for each task.
4. **Prompt Erasing**: at training time, replace the student’s `system instruction`
with a generic one vacated of details of how to approach the task.

For brevity we will not go into technical details. However, we will stop to examine an Orca 2 coding example using the appropriate model from Hugging Face.

### Example code
The following code can be easily executed by harnessing the power of a T4 GPU on Colab.

At first, we install the widely used `transformers` library from Hugging Face.

In [1]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hw2o_xkw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hw2o_xkw
  Resolved https://github.com/huggingface/transformers to commit cbbe30749b425f7c327acdab11473b33567a6e26
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8343539 sha256=5cb042a8ef1ec9823a6cdcc7dcce6bf029999e292bf7c38b33d7868dabb45dd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xnp38ok3/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


The `accelerate` library takes care of the heavy lifting... it's a library that enables the same PyTorch code to be run across any distributed configuration. The `-q` flag in `pip` is used to enable quiet mode during package installation (can be specified up to 3 times to remove messages of increasing levels of importance - warning, error, critical).

In [2]:
!pip install accelerate -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 4.5 MB/s eta 0:00:00


`SentencePiece` is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training ([cit.](https://github.com/google/sentencepiece/blob/master/README.md)) .

In [3]:
!pip install SentencePiece -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00


Protocol buffers (`protobuf`) are Google’s language-neutral, platform-neutral, extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler.

In [4]:
!pip install protobuf -qq

`bitsandbytes` brings quantization to your model. You can now load any pytorch model in 8-bit or 4-bit with a few lines of code.

In [5]:
!pip install bitsandbytes -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00


Import PyTorch and Transformers library of Hugging Face. Set the default PyTorch device to be the GPU if available.

In [6]:
import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_device("cuda")

`transformers.AutoModelForCausalLM.from_pretrained`: this is a method from the Transformers library that loads a pre-trained model. In this case, it's loading a model with causal language modeling capabilities. The term "causal language model" typically refers to a model that generates sequences of tokens in a way that respects causality, where each token is generated based on the preceding tokens.

`microsoft/Orca-2-7b`: this is the identifier for the specific pre-trained model to be loaded. It seems to be a model named "Orca-2-7b" provided by Microsoft.

`device_map='auto'`: this parameter suggests that the code should automatically handle device placement, likely selecting the available GPU if one is present.

`load_in_8bit=True`: this parameter indicates that the model should be loaded using 8-bit quantization, which is a technique to reduce the memory and computational requirements of the model by using lower precision for certain calculations.

In [7]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    "microsoft/Orca-2-7b",
    device_map='auto',
    load_in_8bit=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

Setting the tokenizer: the tokenizer is loaded with the pre-trained model identified by `microsoft/Orca-2-7b`. The `use_fast=False` parameter indicates not to use the fast tokenizer implementation, since fast and slow tokenizers produce different tokens.

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "microsoft/Orca-2-7b",
        use_fast=False,
    )

tokenizer_config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

`system message` is used interchangeably with `system instruction` introduced before.

`tokenizer` tokenizes the combined prompt using the initialized tokenizer. The `return_tensors='pt'` parameter specifies that the output should be PyTorch tensors. The resulting inputs variable holds the tokenized representation of the prompt.

In [9]:
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful?"

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')

After tokenizing the inputs, the `generate()` method returns the generated tokens. The generated tokens then should be converted to text before printing.

The `generate` method of the model is used to generate a sequence of tokens based on the input `input_ids`. The resulting `output_ids` variable holds the generated token IDs.

The `batch_decode` method of the tokenizer is used to convert the generated token IDs (`output_ids`) back into human-readable text. The `[0]` indexing is used to get the decoded text from the first sequence in the batch. Finally, the decoded answer is printed.

In [10]:
output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to find out if a restaurant is popular among locals or mainly attracts tourists, and some possible reasons why this information might be useful are:

- If a restaurant is popular among locals, it might offer better quality, variety, or value for money than a tourist trap, and it might also reflect the local culture, preferences, or traditions.
- If a restaurant is mainly attracting tourists, it might have a more flashy, exotic, or novelty appeal, but it might also charge higher prices, have lower service standards, or offer less

Below, the construction of a second turn message followed by its tokenization. Special tokens are not tokenized: `add_special_tokens`, which defaults to `True`, adds the BOS (beginning of a sentence) token at the beginning and the EOS (end of a sentence) token at the end. If you do not want to use these symbols, you can set `add_special_tokens` to `False`.

Finally, the new input is formed (`second_turn_input`), followed by a new generation and decoding cycle.

In [11]:
# This example continues showing how to add a second turn message by the user to the conversation
second_turn_user_message = "Give me a list of the key points of your first answer."

# we set add_special_tokens=False because we dont want to automatically add a bos_token between messages
second_turn_message_in_markup = f"\n<|im_start|>user\n{second_turn_user_message}<|im_end|>\n<|im_start|>assistant"
second_turn_tokens = tokenizer(second_turn_message_in_markup, return_tensors='pt', add_special_tokens=False)
second_turn_input = torch.cat([output_ids, second_turn_tokens['input_ids']], dim=1)

output_ids_2 = model.generate(second_turn_input,)
second_turn_answer = tokenizer.batch_decode(output_ids_2)[0]

print(second_turn_answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to find out if a restaurant is popular among locals or mainly attracts tourists, and some possible reasons why this information might be useful are:

- If a restaurant is popular among locals, it might offer better quality, variety, or value for money than a tourist trap, and it might also reflect the local culture, preferences, or traditions.
- If a restaurant is mainly attracting tourists, it might have a more flashy, exotic, or novelty appeal, but it might also charge higher prices, have lower service standards, or offer less

In [12]:
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "There are 7 people and 3 apples. How do you cut the apples with at most 4 times to evenly distribute the apples to 7 people?"

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')

output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
There are 7 people and 3 apples. How do you cut the apples with at most 4 times to evenly distribute the apples to 7 people? <|im_end|> 
 <|im_start|> assistant
Given:
- There are 7 people and 3 apples
- The apples need to be cut with at most 4 times
- The apples need to be evenly distributed to 7 people

Goal: Find the number of times to cut the apples

Steps:
- To find the number of times to cut the apples, we need to find the greatest common divisor (GCD) of 7 and 3, which is the largest positive integer that divides both 7 and 3 without leaving a remainder
- One way to find the GCD is to use the Euclidean algorithm, which is based on the fact that GCD(a, b) = GCD(b, a mod b), where a mod b is the remainder of a div

### Conclusions

Orca 2 models, employing diverse reasoning techniques and selecting effective solution strategies for each task, achieve performance levels comparable to, and sometimes surpassing, much larger models, particularly in zero-shot reasoning tasks. Despite inherent limitations, these models exhibit promising potential for future improvement, particularly in terms of enhanced reasoning capabilities, control, and safety through post-training with synthetic data.

Additionally, the potential of using tailored and high-quality synthetic data, generated by more powerful models, for training smaller models in scenarios involving complex prompts and potentially multiple model calls should be underscored.

The authors believe that while frontier models will continue to showcase superior capabilities, research aimed at developing more capable smaller models will open new avenues for applications.

### Useful links

*Orca 2: Teaching Small Language Models How to Reason*
A. Mitra et al. - Microsoft Research
[arXiv:2311.11045](https://arxiv.org/abs/2311.11045) [cs.AI] (2023).

*Orca 2: Teaching Small Language Models How to Reason* -
Microsoft Research Blog ([link](https://www.microsoft.com/en-us/research/blog/orca-2-teaching-small-language-models-how-to-reason/))

*Microsoft Orca-2-7b Model* - Hugging Face [page](https://huggingface.co/microsoft/Orca-2-7b).

